In [1]:
import pyspark.sql as ssql

spark = ssql.SparkSession.builder.master('local') \
    .appName('myAppName') \
    .config('spark.executor.memory', '5gb') \
    .config("spark.cores.max", "6") \
    .getOrCreate()

sc = spark.sparkContext

24/01/06 17:18:57 WARN Utils: Your hostname, luks-note resolves to a loopback address: 127.0.1.1; using 192.168.0.7 instead (on interface wlp0s20f3)
24/01/06 17:18:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/06 17:18:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.parquet("./results_compressed/gerador_residuo_solido.parquet")
# df.printSchema()
# df.show(truncate=False)


In [3]:
rdd = sc.parallelize(df)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyspark/serializers.py", line 459, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 632, in dump
    return Pickler.dump(self, obj)
TypeError: cannot pickle '_thread.RLock' object


PicklingError: Could not serialize object: TypeError: cannot pickle '_thread.RLock' object

In [ ]:
with open("./word_count.sql", "r") as f:
    query = f.read()
query

In [ ]:
query = """
WITH 

splited AS (
    SELECT 

    cast(regexp_replace(regexp_replace(quantidadegerada, '[.]', ''), ',', '.') as double) as quantidadegerada,
    anogeracao,
    CASE 
        WHEN unidade="Litro"      AND classificacaoresiduo="Perigoso"       THEN 'Líquido Perigoso'
        WHEN unidade="Litro"      AND classificacaoresiduo="Não Perigoso"   THEN 'Líquido Não Perigoso'
        WHEN unidade="kilogramas" AND classificacaoresiduo="Perigoso"       THEN 'Sólido Perigoso'
        WHEN unidade="kilogramas" AND classificacaoresiduo="Não Perigoso"   THEN 'Sólido Não Perigoso'
        WHEN unidade="Unidade" AND classificacaoresiduo="Perigoso"       THEN 'Outro Perigoso'
        WHEN unidade="Unidade" AND classificacaoresiduo="Não Perigoso"   THEN 'Outro Não Perigoso'
        ELSE 'Outro'
    END AS tipo,
    split(lower(regexp_replace(detalhe, '[^a-zA-Z0-9áéíóúÁÉÍÓÚàèìòùÀÈÌÒÙäëïöüÄËÏÖÜâêîôûÂÊÎÔÛãñõÃÑÕçÇ°º]', ' ')), ' ') as detalhe,
    split(lower(regexp_replace(tiporesiduo, '[^a-zA-Z0-9áéíóúÁÉÍÓÚàèìòùÀÈÌÒÙäëïöüÄËÏÖÜâêîôûÂÊÎÔÛãñõÃÑÕçÇ°º]', ' ')), ' ') as tiporesiduo,
    split(lower(regexp_replace(categoriaatividade, '[^a-zA-Z0-9áéíóúÁÉÍÓÚàèìòùÀÈÌÒÙäëïöüÄËÏÖÜâêîôûÂÊÎÔÛãñõÃÑÕçÇ°º]', ' ')), ' ') as categoriaatividade,
    split(lower(regexp_replace(razaosocialgerador, '[^a-zA-Z0-9áéíóúÁÉÍÓÚàèìòùÀÈÌÒÙäëïöüÄËÏÖÜâêîôûÂÊÎÔÛãñõÃÑÕçÇ°º]', ' ')), ' ') as razaosocialgerador

    FROM Gerador
),

exp_detalhe AS (
    SELECT 
        word,
        anogeracao,
        tipo,
        SUM(COALESCE(CAST(detalhe AS FLOAT), 0)) AS detalhe
    FROM 
    (
        SELECT 
            explode(detalhe) AS word,
            anogeracao,
            tipo,
            CASE 
                WHEN quantidadegerada IS NULL OR quantidadegerada = '' THEN '0'
                ELSE quantidadegerada
            END AS detalhe
        FROM splited
    ) AS exploded_words
    WHERE NOT (word RLIKE '^[0-9]+([,.][0-9]+)?$') AND word != ''
    GROUP BY word, anogeracao, tipo
    ORDER BY word DESC
),

exp_tiporesiduo AS (
    SELECT 
        word,
        anogeracao,
        tipo,
        SUM(COALESCE(CAST(tiporesiduo AS FLOAT), 0)) AS tiporesiduo
    FROM 
    (
        SELECT 
            explode(tiporesiduo) AS word,
            anogeracao,
            tipo,
            CASE 
                WHEN quantidadegerada IS NULL OR quantidadegerada = '' THEN '0'
                ELSE quantidadegerada
            END AS tiporesiduo
        FROM splited
    ) AS exploded_words
    WHERE NOT (word RLIKE '^[0-9]+([,.][0-9]+)?$') AND word != ''
    GROUP BY word, anogeracao, tipo
    ORDER BY word DESC
),

exp_categoriaatividade AS (
    SELECT 
        word,
        anogeracao,
        tipo,
        SUM(COALESCE(CAST(categoriaatividade AS FLOAT), 0)) AS categoriaatividade
    FROM 
    (
        SELECT 
            explode(categoriaatividade) AS word,
            anogeracao,
            tipo,
            CASE 
                WHEN quantidadegerada IS NULL OR quantidadegerada = '' THEN '0'
                ELSE quantidadegerada
            END AS categoriaatividade
        FROM splited
    ) AS exploded_words
    WHERE NOT (word RLIKE '^[0-9]+([,.][0-9]+)?$') AND word != ''
    GROUP BY word, anogeracao, tipo
    ORDER BY word DESC
),

exp_razaosocialgerador AS (
    SELECT 
        word,
        anogeracao,
        tipo,
        SUM(COALESCE(CAST(razaosocialgerador AS FLOAT), 0)) AS razaosocialgerador
    FROM 
    (
        SELECT 
            explode(razaosocialgerador) AS word,
            anogeracao,
            tipo,
            CASE 
                WHEN quantidadegerada IS NULL OR quantidadegerada = '' THEN '0'
                ELSE quantidadegerada
            END AS razaosocialgerador
        FROM splited
    ) AS exploded_words
    WHERE NOT (word RLIKE '^[0-9]+([,.][0-9]+)?$') AND word != ''
    GROUP BY word, anogeracao, tipo
    ORDER BY word DESC
)

SELECT 
    COALESCE(
        exp_detalhe.word, 
        exp_tiporesiduo.word, 
        exp_categoriaatividade.word, 
        exp_razaosocialgerador.word
        ) AS word,
    COALESCE(
        exp_detalhe.anogeracao, 
        exp_tiporesiduo.anogeracao, 
        exp_categoriaatividade.anogeracao, 
        exp_razaosocialgerador.anogeracao
        ) AS anogeracao,
    COALESCE(
        exp_detalhe.tipo, 
        exp_tiporesiduo.tipo, 
        exp_categoriaatividade.tipo, 
        exp_razaosocialgerador.tipo
        ) AS tipo,  
    COALESCE(detalhe, 0) AS detalhe,
    COALESCE(tiporesiduo, 0) AS tiporesiduo,
    COALESCE(categoriaatividade, 0) AS categoriaatividade,
    COALESCE(razaosocialgerador, 0) AS razaosocialgerador

FROM exp_detalhe 
FULL OUTER JOIN exp_tiporesiduo 
ON exp_detalhe.word = exp_tiporesiduo.word 
AND exp_detalhe.anogeracao = exp_tiporesiduo.anogeracao
AND exp_detalhe.tipo = exp_tiporesiduo.tipo

FULL OUTER JOIN exp_categoriaatividade 
ON COALESCE(exp_detalhe.word, exp_tiporesiduo.word) = exp_categoriaatividade.word
AND COALESCE(exp_detalhe.anogeracao, exp_tiporesiduo.anogeracao) = exp_categoriaatividade.anogeracao
AND COALESCE(exp_detalhe.tipo, exp_tiporesiduo.tipo) = exp_categoriaatividade.tipo

FULL OUTER JOIN exp_razaosocialgerador 
ON COALESCE(exp_detalhe.word, exp_tiporesiduo.word, exp_categoriaatividade.word) = exp_razaosocialgerador.word
AND COALESCE(exp_detalhe.anogeracao, exp_tiporesiduo.anogeracao, exp_categoriaatividade.anogeracao) = exp_razaosocialgerador.anogeracao
AND COALESCE(exp_detalhe.tipo, exp_tiporesiduo.tipo, exp_categoriaatividade.tipo) = exp_razaosocialgerador.tipo

;
"""

In [ ]:
df.printSchema()
df.show(truncate=False)


In [ ]:
sc.sql(query)

In [ ]:
SqlQuery0 = """
WITH 

splited AS (
    SELECT 
    cast(regexp_replace(regexp_replace(quantidadegerada, '[.]', ''), ',', '.') as double) as quantidadegerada,
    anogeracao,
    CASE 
        WHEN unidade="Litro"      AND classificacaoresiduo="Perigoso"       THEN 'Líquido Perigoso'
        WHEN unidade="Litro"      AND classificacaoresiduo="Não Perigoso"   THEN 'Líquido Não Perigoso'
        WHEN unidade="kilogramas" AND classificacaoresiduo="Perigoso"       THEN 'Sólido Perigoso'
        WHEN unidade="kilogramas" AND classificacaoresiduo="Não Perigoso"   THEN 'Sólido Não Perigoso'
        WHEN unidade="Unidade" AND classificacaoresiduo="Perigoso"       THEN 'Outro Perigoso'
        WHEN unidade="Unidade" AND classificacaoresiduo="Não Perigoso"   THEN 'Outro Não Perigoso'
        ELSE 'Outro'
    END AS tipo,
    split(lower(regexp_replace(detalhe, '[^a-zA-Z0-9áéíóúÁÉÍÓÚàèìòùÀÈÌÒÙäëïöüÄËÏÖÜâêîôûÂÊÎÔÛãñõÃÑÕçÇ°º]', ' ')), ' ') as detalhe,
    split(lower(regexp_replace(tiporesiduo, '[^a-zA-Z0-9áéíóúÁÉÍÓÚàèìòùÀÈÌÒÙäëïöüÄËÏÖÜâêîôûÂÊÎÔÛãñõÃÑÕçÇ°º]', ' ')), ' ') as tiporesiduo,
    split(lower(regexp_replace(categoriaatividade, '[^a-zA-Z0-9áéíóúÁÉÍÓÚàèìòùÀÈÌÒÙäëïöüÄËÏÖÜâêîôûÂÊÎÔÛãñõÃÑÕçÇ°º]', ' ')), ' ') as categoriaatividade,
    split(lower(regexp_replace(razaosocialgerador, '[^a-zA-Z0-9áéíóúÁÉÍÓÚàèìòùÀÈÌÒÙäëïöüÄËÏÖÜâêîôûÂÊÎÔÛãñõÃÑÕçÇ°º]', ' ')), ' ') as razaosocialgerador

    FROM myDataSource
),

exp_detalhe AS (
    SELECT 
        word,
        anogeracao,
        tipo,
        SUM(COALESCE(CAST(detalhe AS FLOAT), 0)) AS detalhe
    FROM 
    (
        SELECT 
            explode(detalhe) AS word,
            anogeracao,
            tipo,
            CASE 
                WHEN quantidadegerada IS NULL OR quantidadegerada = '' THEN '0'
                ELSE quantidadegerada
            END AS detalhe
        FROM splited
    ) AS exploded_words
    WHERE NOT (word RLIKE '^[0-9]+([,.][0-9]+)?$') AND word != ''
    GROUP BY word, anogeracao, tipo
    ORDER BY word DESC
),

exp_tiporesiduo AS (
    SELECT 
        word,
        anogeracao,
        tipo,
        SUM(COALESCE(CAST(tiporesiduo AS FLOAT), 0)) AS tiporesiduo
    FROM 
    (
        SELECT 
            explode(tiporesiduo) AS word,
            anogeracao,
            tipo,
            CASE 
                WHEN quantidadegerada IS NULL OR quantidadegerada = '' THEN '0'
                ELSE quantidadegerada
            END AS tiporesiduo
        FROM splited
    ) AS exploded_words
    WHERE NOT (word RLIKE '^[0-9]+([,.][0-9]+)?$') AND word != ''
    GROUP BY word, anogeracao, tipo
    ORDER BY word DESC
),

exp_categoriaatividade AS (
    SELECT 
        word,
        anogeracao,
        tipo,
        SUM(COALESCE(CAST(categoriaatividade AS FLOAT), 0)) AS categoriaatividade
    FROM 
    (
        SELECT 
            explode(categoriaatividade) AS word,
            anogeracao,
            tipo,
            CASE 
                WHEN quantidadegerada IS NULL OR quantidadegerada = '' THEN '0'
                ELSE quantidadegerada
            END AS categoriaatividade
        FROM splited
    ) AS exploded_words
    WHERE NOT (word RLIKE '^[0-9]+([,.][0-9]+)?$') AND word != ''
    GROUP BY word, anogeracao, tipo
    ORDER BY word DESC
),

exp_razaosocialgerador AS (
    SELECT 
        word,
        anogeracao,
        tipo,
        SUM(COALESCE(CAST(razaosocialgerador AS FLOAT), 0)) AS razaosocialgerador
    FROM 
    (
        SELECT 
            explode(razaosocialgerador) AS word,
            anogeracao,
            tipo,
            CASE 
                WHEN quantidadegerada IS NULL OR quantidadegerada = '' THEN '0'
                ELSE quantidadegerada
            END AS razaosocialgerador
        FROM splited
    ) AS exploded_words
    WHERE NOT (word RLIKE '^[0-9]+([,.][0-9]+)?$') AND word != ''
    GROUP BY word, anogeracao, tipo
    ORDER BY word DESC
)

SELECT 
    COALESCE(
        exp_detalhe.word, 
        exp_tiporesiduo.word, 
        exp_categoriaatividade.word, 
        exp_razaosocialgerador.word
        ) AS word,
    COALESCE(
        exp_detalhe.anogeracao, 
        exp_tiporesiduo.anogeracao, 
        exp_categoriaatividade.anogeracao, 
        exp_razaosocialgerador.anogeracao
        ) AS anogeracao,
    COALESCE(
        exp_detalhe.tipo, 
        exp_tiporesiduo.tipo, 
        exp_categoriaatividade.tipo, 
        exp_razaosocialgerador.tipo
        ) AS tipo,  
    COALESCE(detalhe, 0) AS detalhe,
    COALESCE(tiporesiduo, 0) AS tiporesiduo,
    COALESCE(categoriaatividade, 0) AS categoriaatividade,
    COALESCE(razaosocialgerador, 0) AS razaosocialgerador

FROM exp_detalhe 
FULL OUTER JOIN exp_tiporesiduo 
ON exp_detalhe.word = exp_tiporesiduo.word 
AND exp_detalhe.anogeracao = exp_tiporesiduo.anogeracao
AND exp_detalhe.tipo = exp_tiporesiduo.tipo

FULL OUTER JOIN exp_categoriaatividade 
ON COALESCE(exp_detalhe.word, exp_tiporesiduo.word) = exp_categoriaatividade.word
AND COALESCE(exp_detalhe.anogeracao, exp_tiporesiduo.anogeracao) = exp_categoriaatividade.anogeracao
AND COALESCE(exp_detalhe.tipo, exp_tiporesiduo.tipo) = exp_categoriaatividade.tipo

FULL OUTER JOIN exp_razaosocialgerador 
ON COALESCE(exp_detalhe.word, exp_tiporesiduo.word, exp_categoriaatividade.word) = exp_razaosocialgerador.word
AND COALESCE(exp_detalhe.anogeracao, exp_tiporesiduo.anogeracao, exp_categoriaatividade.anogeracao) = exp_razaosocialgerador.anogeracao
AND COALESCE(exp_detalhe.tipo, exp_tiporesiduo.tipo, exp_categoriaatividade.tipo) = exp_razaosocialgerador.tipo

;
"""

In [ ]:
sc.from_options(
    
)

In [ ]:
Gerador = sc.create_dynamic_frame.from_options(
    format_options={},
    connection_type="s3",
    format="parquet",
    connection_options={
        "paths": [
            "s3://jm2vtech/pos/residuos_solidos_ibama/results_compressed/gerador_residuo_solido.parquet"
        ],
        "recurse": True,
    },
    transformation_ctx="Gerador",
)

In [ ]:
print(SqlQuery0.replace("myDataSource", "Gerador"))

In [ ]:
DynamicFrame.fromDF(SqlQuery0.replace("myDataSource", "Gerador"), sc, transformation_ctx)

In [ ]:
def sparkSqlQuery(glueContext, query, mapping, transformation_ctx) -> DynamicFrame:
    for alias, frame in mapping.items():
        frame.toDF().createOrReplaceTempView(alias)
    result = spark.sql(query)
    return DynamicFrame.fromDF(result, glueContext, transformation_ctx)

In [ ]:
text_split_map_reduce = sparkSqlQuery(
    glueContext,
    query=SqlQuery0,
    mapping={"myDataSource": Gerador},
    transformation_ctx="text_split_map_reduce",
)